### 1-Supply Chain Project Stages:-

1- Data Understanding (Understand what each column represent).

2- Data Loading.

3- Data Exploration (Overview about the data).

4- Data Cleaning.

5- Feature Engineering.

6- Data Analysis.

7-Data Preprocessing.

# 1-Data Understanding

### Import Necessary Libraries

In [97]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np

import streamlit as st
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from category_encoders import BinaryEncoder

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE



# 2-Data Loading

In [ ]:
import pandas as pd

df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='ISO-8859-1') 
df



# 3-Data Exploration

In [ ]:
# Check Data Types 
# Dataset Information
df.info()

In [258]:
# Dataset Shape (Rows, Columns)
df.shape

(180519, 53)

In [ ]:
# Preview First 5 Rows
df.head()

In [34]:
#  Column Names
df.columns

Index(['Type', 'Days_for_shipping_real', 'Days_for_shipment_scheduled',
       'Benefit_per_order', 'Sales_per_customer', 'Delivery_Status',
       'Late_delivery_risk', 'Category_Id', 'Category_Name', 'Customer_City',
       'Customer_Country', 'Customer_Fname', 'Customer_Lname',
       'Customer_Segment', 'Customer_State', 'Department_Id',
       'Department_Name', 'Latitude', 'Longitude', 'Market', 'Order_City',
       'Order_Country', 'order_date_DateOrders', 'Order_Item_Discount',
       'Order_Item_Discount_Rate', 'Order_Item_Product_Price',
       'Order_Item_Profit_Ratio', 'Order_Item_Quantity', 'Sales',
       'Order_Item_Total', 'Order_Profit_Per_Order', 'Order_Region',
       'Order_State', 'Order_Status', 'Product_Name', 'Product_Price',
       'Product_Status', 'shipping_date_DateOrders', 'Shipping_Mode'],
      dtype='object')

In [35]:
# Data Types of Each Column
df.dtypes

Type                            object
Days_for_shipping_real           int64
Days_for_shipment_scheduled      int64
Benefit_per_order              float64
Sales_per_customer             float64
Delivery_Status                 object
Late_delivery_risk               int64
Category_Id                      int64
Category_Name                   object
Customer_City                   object
Customer_Country                object
Customer_Fname                  object
Customer_Lname                  object
Customer_Segment                object
Customer_State                  object
Department_Id                    int64
Department_Name                 object
Latitude                       float64
Longitude                      float64
Market                          object
Order_City                      object
Order_Country                   object
order_date_DateOrders           object
Order_Item_Discount            float64
Order_Item_Discount_Rate       float64
Order_Item_Product_Price 

In [ ]:
# Summary Statistics for Numerical Columns
df.describe(include='number')


In [ ]:
# Summary Statistics for Categorical Columns
df.describe(include='object')


In [ ]:
#  Number of Unique Values per Column
df.nunique()

In [265]:
# Check for Duplicates
df.duplicated().sum()


0

In [ ]:
# Check for Missing Values
df.isnull().sum()


In [ ]:
# Percentage of Missing Values per Column
df.isnull().mean() * 100

# 4-Data Cleaning
- Drop Duplicate Rows
- Handle Missing Values
- In-depth Check for Numerical Columns
- Handle Outliers
- Clean String Columns
- Standardize Column Names
- Remove Placeholder Values
- In-depth Check for Categorical Columns
- Remove Irrelevant Columns
- Reset the index after cleaning

Drop Duplicate Rows

In [18]:
# Remove any duplicate records from the dataset
df = df.drop_duplicates()

 Handle Missing Values

In [21]:
# Calculate Missing Values Percentage for Each Column
missing_percent = df.isnull().mean() * 100

In [22]:
# Drop rows if missing percentage < 5%
drop_na_cols = missing_percent[missing_percent < 5].index.tolist()
df.dropna(subset=drop_na_cols, inplace=True)

In [23]:
# Drop columns if missing percentage > 40%
drop_cols = missing_percent[missing_percent > 40].index.tolist()
df.drop(columns=drop_cols, inplace=True)

In-depth Check for Numerical Columns

In [ ]:
# Identify numerical columns
num_cols = df.select_dtypes(include='number').columns.tolist()

# Visual inspection using Histograms
import plotly.express as px

for col in num_cols:
    px.histogram(data_frame=df, x=col, title=col).show()


In [ ]:
# Summary statistics
df[num_cols].describe()

In [ ]:
# Unique values
df[num_cols].nunique()

In [275]:
# Check for negative values
for col in num_cols:
    if (df[col] < 0).any():
        print(f"{col} contains negative values")


Order Profit Per Order contains negative values


In [276]:
# Boxplot visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
for i, col in enumerate(num_cols[:6]):
    plt.subplot(2, 3, i+1)
    sns.boxplot(data=df, y=col)
    plt.title(col)
plt.tight_layout()
plt.show()


Handle Outliers

In [277]:
# Extract only numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Number of rows before cleaning
print("Number of rows before removing outliers:", df.shape[0])

# Use IQR to remove outliers
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# Number of rows after cleaning
print("Number of rows after removing outliers:", df.shape[0])


Number of rows before removing outliers: 180508
Number of rows after removing outliers: 121137


Clean String Columns

In [43]:
# Clean String Columns
clean_cols = ['Customer City', 'Customer Country', 'Order Status', 'Order Region', 'Category Name']
for col in clean_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()


Standardize Column Names

In [24]:
# Standardize Column Names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace(r'[()]', '', regex=True)

Remove Placeholder Values

In [ ]:
# Remove Placeholder Values
placeholders = ['xxxxxxxxx', 'unknown', 'n/a', 'none']
cols_to_check = ['Customer_Email', 'Customer_Password']

for col in cols_to_check:
    df[col] = df[col].astype(str).str.lower().replace(placeholders, np.nan)

df.dropna(subset=cols_to_check, inplace=True)
df.reset_index(drop=True, inplace=True)


In-depth Check for Categorical Columns

In [ ]:
# Display Top 3 Categories in Each Categorical Column
cat_cols = df.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    print(f"\nTop categories in '{col}':")
    print(df[col].value_counts().head(3))


In [ ]:
# Count of Unique Values Per Categorical Column
for col in cat_cols:
    print(col)
    print("Unique Count:", df[col].nunique())
    print("Unique Values:", df[col].unique())
    print('-' * 100)


In [ ]:
# Dominant ratio in each column
for col in cat_cols:
    ratio = df[col].value_counts(normalize=True).max()
    print(col)
    print(f"Dominant Category Ratio: {ratio:.2%}")
    print('-' * 100)


Remove Columns Not Relevant for Analysis

In [29]:
# Remove Irrelevant Columns
columns_to_drop = [
    'Customer_Id',
    'Customer_Email',
    'Customer_Password',
    'Customer_Street',
    'Customer_Zipcode',
    'Order_Id',
    'Order_Customer_Id',
    'Order_Item_Id',
    'Order_Item_Cardprod_Id',
    'Product_Card_Id',
    'Product_Category_Id',
    'Product_Image',
    'Product_Description'
]

columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df.drop(columns=columns_to_drop, inplace=True)


Redefine numerical columns after cleaning

In [ ]:
# Redefine numerical columns after cleaning
num_cols = df.select_dtypes(include='number').columns.tolist()

# Final Histogram plotting after dropping irrelevant columns and outliers
import plotly.express as px
for col in num_cols:
    px.histogram(data_frame=df, x=col, title=f"Histogram of {col}").show()


# 5-FEATURE ENGINEERING:-

 -DOMAIN KNOWLEDGE FEATURES.

 -DATE AND TIME FEATURES.

 -PRICING & VALUE FLAGS.

 -STRING FEATURES.

 -GEOSPATIAL FEATURES.


### DOMAIN KNOWLEDGE FEATURES

In [100]:
# Profitability_Flag
df['Is_Profitable_Order'] = (df['Order Profit Per Order'] > 0).astype(int)

# Zero_Profit_Flag
df['Is_Zero_Profit'] = (df['Order Profit Per Order'] == 0).astype(int)

# Profit_Ratio
df['Order_Item_Profit_Ratio'] = df['Order Profit Per Order'] / df['Order Item Total']

# Profit_Margin_Copy
df['Profit_Margin'] = df['Order_Item_Profit_Ratio']

# Profit_Category_Binning
df['Profitability_Category'] = pd.cut(df['Profit_Margin'], bins=[-1, 0, 0.2, 0.5, 1], labels=['Loss', 'Low', 'Medium', 'High'])

# Low_Profit_High_Sales_Flag
df['Low_Profit_High_Sales'] = ((df['Profit_Margin'] < 0.1) & (df['Order Item Total'] > 500)).astype(int)

# Order_Value_Binning
df['Order_Value_Category'] = pd.cut(df['Order Item Total'], bins=[0, 100, 500, 1000, float('inf')], labels=['Low', 'Medium', 'High', 'Very High'])

# Customer_Segment_Binning
df['Customer_Segment'] = pd.cut(df['Sales per customer'], bins=[0, 100, 500, 1000, np.inf], labels=['Low', 'Medium', 'High', 'Very High'])

# Order_Quarter_Extraction
df['Order_Quarter'] = pd.to_datetime(df['order date (DateOrders)']).dt.to_period('Q')

# Custom_Profit_Level
def classify_profit(ratio):
    if ratio < 0.2:
        return 'low'
    elif ratio <= 0.5:
        return 'medium'
    else:
        return 'high'
df['Profit_Category'] = df['Order_Item_Profit_Ratio'].apply(classify_profit)

# Order_Type_Classification
df['Order_Type'] = np.where((df['Order Item Product Price'] > 1000) & (df['Order Item Discount'] < 100), 'premium', 'regular')


 ### DATE AND TIME FEATURES

In [101]:
# Extract_Order_Year
df['Order_Year'] = pd.to_datetime(df['order date (DateOrders)']).dt.year

# Extract_Order_Month
df['Order_Month'] = pd.to_datetime(df['order date (DateOrders)']).dt.month

# Extract_Order_Weekday
df['Order_Weekday'] = pd.to_datetime(df['order date (DateOrders)']).dt.day_name()

# Weekend_Flag
df['Order_Weekend'] = df['Order_Weekday'].isin(['Saturday', 'Sunday']).astype(int)

# Shipping_Duration_Days
df['Shipping_Duration'] = (pd.to_datetime(df['shipping date (DateOrders)']) - pd.to_datetime(df['order date (DateOrders)'])).dt.days


### PRICING & VALUE FLAGS

In [103]:
# Price_After_Discount
df['Price_After_Discount'] = df['Order Item Product Price'] - df['Order Item Discount']

# Discount_Level_Binning
df['Discount_Level'] = pd.cut(df['Order Item Discount Rate'], bins=[-0.01, 0, 0.2, 0.5, 1], labels=['No Discount', 'Low', 'Medium', 'High'])

# High_Value_Order_Flag
df['Is_High_Value_Order'] = (df['Order Item Product Price'] > 500).astype(int)

# Total_Discount_Amount
df['Total_Discount'] = df['Order Item Discount'] * df['Order Item Quantity']

# Unit_Revenue
df['Unit_Revenue'] = df['Order Item Product Price'] * df['Order Item Quantity']

# Total_Units_Ordered
df['Total_Units_Ordered'] = df['Order Item Quantity']

# Average_Unit_Price
df['Avg_Price_per_Unit'] = df['Order Item Product Price'] / df['Order Item Quantity']

# Has_Discount_Flag
df['Has_Discount'] = (df['Order Item Discount'] > 0).astype(int)


###  STRING FEATURES

In [47]:
# Full_Customer_Name
df['Customer_FullName'] = df['Customer Fname'] + ' ' + df['Customer Lname']

# Customer_Location_Concat
df['Customer_Location'] = df['Customer City'].astype(str) + ', ' + df['Customer Country'].astype(str)

# Customer_Name_Token_Count
df['Customer_Name_Tokens'] = df['Customer_FullName'].astype(str).apply(lambda x: len(x.split()))

# Product_Name_Word_Count
df['Product_Name_WordCount'] = df['Product Name'].astype(str).apply(lambda x: len(x.split()))


In [48]:
# Customer_Purchase_Frequency
df['Customer_Frequency'] = df['Customer_FullName'].map(df['Customer_FullName'].value_counts())

### GEOSPATIAL FEATURES

In [105]:
# Country_Region_Mapping_Function
def country_group(country):
    country = str(country).lower()
    if 'us' in country or 'america' in country:
        return 'North America'
    elif 'germany' in country or 'uk' in country or 'france' in country:
        return 'Europe'
    elif 'india' in country or 'china' in country:
        return 'Asia'
    else:
        return 'Other'

# Country_Group_Assignment
df['Country_Group'] = df['Customer Country'].apply(country_group)

# Major_City_Flag
df['Is_Major_City'] = df['Customer City'].astype(str).str.lower().isin(['new york', 'los angeles', 'london', 'paris', 'berlin', 'dubai']).astype(int)


# 6-Data Analysis

### Univariate

What is the distribution of Benefit per order?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Benefit per order', nbins=30, title='Histogram of Benefit per order')
fig.show()

df['Benefit per order'].describe()

In [ ]:

plt.figure()
sns.histplot(df['Benefit per order'], kde=True, bins=30)
plt.title('Histogram of Benefit per order')
plt.xlabel('Benefit per order')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


What is the distribution of Sales per customer?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Sales per customer', nbins=30, title='Histogram of Sales per customer')
fig.show()

df['Sales per customer'].describe()

In [ ]:
#Box Plot of Sales per Customer

import plotly.express as px

px.box(data_frame=df, y='Sales per customer', title='Box Plot of Sales per Customer')


What is the distribution of Order Item Total?

In [ ]:
import plotly.express as px

fig = px.box(data_frame=df, y='Order Item Total', title='Distribution of Order Item Total')
fig.show()

df['Order Item Total'].describe()


What is the distribution of Order Item Profit?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Order Profit Per Order', nbins=30, title='Histogram of Order Profit Per Order')
fig.show()

df['Order Profit Per Order'].describe()

In [ ]:
plt.figure()
sns.histplot(df['Order Profit Per Order'], kde=True, bins=30)
plt.title('Histogram of Order Profit Per Order')
plt.xlabel('Order Profit Per Order')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

What is the distribution of Order Item Profit Ratio?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Order Item Profit Ratio', nbins=30, title='Histogram of Order Item Profit Ratio')
fig.show()

df['Order Item Profit Ratio'].describe()


What is the average profit margin for each profitability category?

In [69]:

avg_profit = df.groupby('Profitability_Category')['Profit_Margin'].mean().reset_index()

fig = px.pie(
    data_frame=avg_profit,
    names='Profitability_Category',
    values='Profit_Margin',
    title='Average Profit Margin per Profitability Category',
    color_discrete_sequence=px.colors.qualitative.Set2  
)

fig.show()


C:\Users\maria\AppData\Local\Temp\ipykernel_2280\3651272988.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




How are orders spread across the different profitability levels?

In [ ]:
# Pie Chart for Profitability_Category

import plotly.express as px

fig = px.pie(
    data_frame=df,
    names='Profitability_Category',
    title='Distribution of Profitability Category',
    color_discrete_sequence=px.colors.qualitative.Set2  # ✅ مجموعة ألوان مميزة
)

fig.show()


How are profit margins spread across the orders?

In [ ]:
import plotly.express as px

# Box Plot to analyze the distribution of Profit Margin
fig = px.box(data_frame=df, y='Profit_Margin', title='Distribution of Profit Margin')
fig.show()

df['Profit_Margin'].describe()

What is the distribution of Order Item Discount Rate?

In [ ]:

import plotly.express as px

fig = px.histogram(df, x='Order Item Discount Rate', nbins=30, title='Histogram of Order Item Discount Rate')
fig.show()

df['Order Item Discount Rate'].describe()


What is the distribution of Late_delivery_risk?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Late_delivery_risk', nbins=30, title='Histogram of Late_delivery_risk')
fig.show()

df['Late_delivery_risk'].describe()


How are customers divided into different segments?

In [ ]:
fig = px.pie(
    data_frame=df,
    names='Customer_Segment',
    title='Distribution of Customer Segment',
    color_discrete_sequence=px.colors.qualitative.Set2  
)

fig.show()


What is the most common request type

In [77]:
import plotly.express as px

type_counts = df['Type'].value_counts().reset_index()
type_counts.columns = ['Order Type', 'Count']

fig = px.bar(type_counts, x='Order Type', y='Count', title='Order Type Frequency')
fig.show()

df['Type'].describe()

count     180519
unique         4
top        DEBIT
freq       69295
Name: Type, dtype: object

What are the most common categories in 'Order Status'?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Order Status', title='Bar Chart of Order Status')
fig.update_layout(xaxis_title='Order Status', yaxis_title='Count', xaxis_tickangle=-45)
fig.show()

df['Order Status'].describe()

What are the most common categories in 'Order Region'?

In [ ]:
import plotly.express as px

region_counts = df['Order Region'].value_counts().reset_index()
region_counts.columns = ['Order Region', 'Count']

fig = px.bar(region_counts, x='Order Region', y='Count', title='Bar Chart of Order Region')
fig.update_layout(xaxis_tickangle=-45)  
fig.show()

df['Order Region'].describe()

What are the most common categories in 'Delivery Status'?

In [ ]:
import plotly.express as px

fig = px.histogram(df, x='Delivery Status', title='Bar Chart of Delivery Status')
fig.update_layout(xaxis_title='Delivery Status', yaxis_title='Count', xaxis_tickangle=-45)
fig.show()

df['Delivery Status'].describe()


### Bivariate 

What is the correlation among all numerical variables?

In [ ]:
import plotly.express as px
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
fig = px.imshow(df[numerical_cols].corr(), text_auto=True, color_continuous_scale='RdBu_r', title='Correlation Heatmap')
fig.show()


In [ ]:
sns.heatmap(df[numerical_cols].corr(), annot=True, cmap='coolwarm')

What is the relationship between Sales per Customer and Profit Margin?

In [ ]:
import plotly.express as px

fig = px.scatter(df, x='Sales per customer', y='Profit_Margin', title='Sales per Customer vs Profit Margin')
fig.show()


How does Profit Margin vary across Profitability Categories?

In [ ]:
import plotly.express as px

fig = px.box(df, x='Profitability_Category', y='Profit_Margin', title='Profit Margin by Profitability Category')
fig.show()


How does Sales per Customer vary by Customer Segment?

In [ ]:
import plotly.express as px

fig = px.box(df, x='Customer Segment', y='Sales per customer', title='Sales per Customer by Segment')
fig.show()


How does Order Item Total differ by Order Status?

In [ ]:
import plotly.express as px

fig = px.violin(df, x='Order Status', y='Order Item Total', box=True, points='all', title='Order Item Total by Order Status')
fig.show()


What is the distribution of Benefit per Order by Order Region?

In [ ]:
import plotly.express as px

fig = px.strip(
    df,
    x='Order Region',
    y='Benefit per order',
    color='Order Region',  
    title='Benefit per Order by Order Region'
)

fig.show()



Does Order Item Quantity increase with Order Item Total?

In [ ]:
import plotly.express as px

fig = px.scatter(df, x='Order Item Quantity', y='Order Item Total', title='Order Item Total vs Order Item Quantity')
fig.show()


How does Discount Level affect Sales per Customer?

In [ ]:
import plotly.express as px

fig = px.box(df, x='Discount_Level', y='Sales per customer', title='Sales per Customer by Discount Level')
fig.show()



Does Discount Rate affect Profit Ratio?

In [ ]:
import plotly.express as px

fig = px.scatter(
    df,
    x='Order Item Discount Rate',
    y='Order Item Profit Ratio',
    title='Discount Rate vs Profit Ratio'
)
fig.show()


 Is Profit Ratio consistent across Regions?

In [ ]:
import plotly.express as px

fig = px.violin(
    df,
    x='Order Region',
    y='Order Item Profit Ratio',
    box=True,              
    points='all',          
    title='Profit Ratio by Region'
)
fig.show()


Do different Order Statuses impact Profit Margin?

In [ ]:
import plotly.express as px

fig = px.violin(df, x='Order Status', y='Profit_Margin', box=True, points='all')
fig.show()


What is the variation of Benefit per Order by Customer Segment?

In [ ]:
import plotly.express as px

fig = px.strip(df, x='Customer Segment', y='Benefit per order')
fig.show()


How does Order Item Quantity differ across Product Categories?

In [ ]:
import plotly.express as px

fig = px.box(df, x='Category Name', y='Order Item Quantity')
fig.show()


How does Profit Margin change over Order Item Total ?

In [ ]:
import plotly.express as px

df_sorted = df.sort_values('Order Item Total')

fig = px.line(df_sorted, x='Order Item Total', y='Profit_Margin')
fig.show()


Are high-value orders always more profitable?

In [ ]:
import plotly.express as px

fig = px.box(df, x='Order_Value_Category', y='Order Profit Per Order', title='Profit by Order Value Category')
fig.show()


### MULTIVARIATE ANALYSIS

How do Discount Level, Order Value Category and Profit relate together?

In [ ]:
import plotly.express as px

fig = px.box(
    df,
    x='Discount_Level',
    y='Order Profit Per Order',
    color='Order_Value_Category',
    title='Profit by Discount Level and Order Value Category'
)
fig.show()


Does Region and Order Type influence Profit Ratio?

In [ ]:
import plotly.express as px

fig = px.box(
    df,
    x='Order Region',
    y='Order_Item_Profit_Ratio',
    color='Order_Type',
    title='Profit Ratio by Region and Order Type'
)
fig.show()


How does Month and Segment affect Total Sales?

In [ ]:
import plotly.express as px

fig = px.bar(
    df,
    x='Order_Month',
    y='Order Item Total',
    color='Customer_Segment',
    title='Monthly Sales by Customer Segment',
    barmode='group'
)
fig.show()


# 7-Data Preprocessing


Step1. Split Data into Input Features and Target Variable.

Step2. Split Data into Train and Test.

Step3. Initial Data Inspection

Step4. Nominal Encoding

Step5. Binary Encoding

Step6. Data Visualization

Step7. Impute Missing

Step8. Numerical Scaling

Step9. Handle Class Imbalance

Step10. Model Training

Step11. Model Evaluation

In [3]:
# 1. Split data into features and target
x = df.drop('Late_delivery_risk', axis=1)
y = df['Late_delivery_risk']

In [4]:
# 2. Split data into training and testing sets 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
x_train.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [5]:
# 3. Initial Data Inspection
# Show number of rows and columns + display first 10 rows of training data
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
x_train.head(10)

x_train shape: (126363, 52)
y_train shape: (126363,)


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Category Id,Category Name,Customer City,Customer Country,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,TRANSFER,2,1,11.640000,38.790001,Late delivery,29,Shop By Sport,Brooklyn,EE. UU.,...,NaN,627,29,NaN,http://images.acmesports.sports/Under+Armour+G...,Under Armour Girls' Toddler Spine Surge Runni,39.990002,0,4/2/2017 4:06,First Class
1,CASH,2,4,82.889999,176.369995,Advance shipping,17,Cleats,Caguas,Puerto Rico,...,NaN,365,17,NaN,http://images.acmesports.sports/Perfect+Fitnes...,Perfect Fitness Perfect Rip Deck,59.990002,0,8/11/2015 8:27,Standard Class
2,DEBIT,2,4,88.720001,272.980011,Advance shipping,43,Camping & Hiking,Caguas,Puerto Rico,...,NaN,957,43,NaN,http://images.acmesports.sports/Diamondback+Wo...,Diamondback Women's Serene Classic Comfort Bi,299.980011,0,4/3/2015 1:37,Standard Class
3,TRANSFER,6,4,74.089996,227.960007,Late delivery,17,Cleats,Miami,EE. UU.,...,98103.0,365,17,NaN,http://images.acmesports.sports/Perfect+Fitnes...,Perfect Fitness Perfect Rip Deck,59.990002,0,5/25/2016 6:18,Standard Class
4,DEBIT,3,4,45.500000,129.990005,Advance shipping,18,Men's Footwear,Caguas,Puerto Rico,...,NaN,403,18,NaN,http://images.acmesports.sports/Nike+Men%27s+C...,Nike Men's CJ Elite 2 TD Football Cleat,129.990005,0,10/9/2015 18:10,Standard Class
5,TRANSFER,0,0,36.619999,116.989998,Shipping on time,18,Men's Footwear,Massillon,EE. UU.,...,NaN,403,18,NaN,http://images.acmesports.sports/Nike+Men%27s+C...,Nike Men's CJ Elite 2 TD Football Cleat,129.990005,0,3/29/2016 18:11,Same Day
6,PAYMENT,3,4,36.400002,363.980011,Advance shipping,45,Fishing,Hanford,EE. UU.,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,12/29/2015 6:19,Standard Class
7,DEBIT,6,4,178.550003,371.980011,Late delivery,45,Fishing,Gardena,EE. UU.,...,NaN,1004,45,NaN,http://images.acmesports.sports/Field+%26+Stre...,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,0,6/27/2017 22:12,Standard Class
8,TRANSFER,3,4,64.019997,145.500000,Advance shipping,24,Women's Apparel,Winter Park,EE. UU.,...,77095.0,502,24,NaN,http://images.acmesports.sports/Nike+Men%27s+D...,Nike Men's Dri-FIT Victory Golf Polo,50.000000,0,7/2/2016 16:13,Standard Class
9,CASH,4,2,1.360000,113.089996,Late delivery,18,Men's Footwear,Brooklyn,EE. UU.,...,NaN,403,18,NaN,http://images.acmesports.sports/Nike+Men%27s+C...,Nike Men's CJ Elite 2 TD Football Cleat,129.990005,0,7/3/2015 19:11,Second Class


In [6]:
# 4. Nominal - One-Hot Encode the 'Order Status' column
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_cols = ['Order Status']
ohe_train_array = ohe.fit_transform(x_train[ohe_cols])
ohe_test_array = ohe.transform(x_test[ohe_cols])
ohe_feature_names = ohe.get_feature_names_out(ohe_cols)
ohe_train_df = pd.DataFrame(ohe_train_array, columns=ohe_feature_names)
ohe_test_df = pd.DataFrame(ohe_test_array, columns=ohe_feature_names)
x_train = pd.concat([x_train, ohe_train_df], axis=1).drop(ohe_cols, axis=1)
x_test = pd.concat([x_test, ohe_test_df], axis=1).drop(ohe_cols, axis=1)

In [7]:
# 5. Binary encode the 'Customer State'.
if 'Customer State' in x_train.columns:
    be = BinaryEncoder()
    be_train_df = be.fit_transform(x_train[['Customer State']])
    be_test_df = be.transform(x_test[['Customer State']])
    x_train = pd.concat([x_train, be_train_df], axis=1).drop('Customer State', axis=1)
    x_test = pd.concat([x_test, be_test_df], axis=1).drop('Customer State', axis=1)

In [8]:
# 6. Data Visualization
# Visualize the distribution of 'Order Profit Per Order' column (make sure the column exists)
fig = px.histogram(data_frame=x_train, x='Order Profit Per Order')
fig.show()

In [9]:
# 7. Impute Missing - Fill missing values in 'Order Item Total' column
num_col = 'Order Item Total'
x_train_num = x_train[[num_col]]
x_test_num = x_test[[num_col]]

imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train_num)
x_test_imputed = imputer.transform(x_test_num)

In [10]:
# 8. Numerical - Scale the 'Order Item Total' column
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_imputed)
x_test_scaled = scaler.transform(x_test_imputed)

In [11]:
# 9. Handle class imbalance using SMOTE
smote = SMOTE(random_state=40)
x_train_balanced, y_train_balanced = smote.fit_resample(x_train_scaled, y_train)

C:\Users\maria\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:474: FutureWarning:

`BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.



In [12]:
# 10. Train Logistic Regression model
model = LogisticRegression()
model.fit(x_train_balanced, y_train_balanced)

LogisticRegression()

In [13]:
# 11. Evaluate the model
y_pred = model.predict(x_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.42      0.44     24478
           1       0.55      0.58      0.56     29678

    accuracy                           0.51     54156
   macro avg       0.50      0.50      0.50     54156
weighted avg       0.51      0.51      0.51     54156



### Data Preprocessing using pipline (alternative option)

Step1.Split Data into Input Features and Target Variable.

Step2. Split Data into Train and Test.

Step3.Create Numerical & Categorical Pipelines.

Step4.Column Transformer to Assign columns to be processed.

Step5.Model Definition and Training.

Step6.Model Evaluation.

Step7.Cross Validation & Hyperparameter Tuning.

Step1. Split Data into Input Features and Target Variable

In [22]:
x = df.drop('Late_delivery_risk', axis=1)
y = df['Late_delivery_risk']

Step2. Split Data into Train and Test

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [ ]:
px.histogram(data_frame= x_train, x= 'Order Profit Per Order')

Step3. Create Numerical & Categorical Pipelines


In [24]:
# Define Columns (Numerical, Nominal, Ordinal, Binary)
num_cols = ['Order Profit Per Order', 'Order Item Total', 'Sales per customer',
            'Order Item Discount', 'Order Item Discount Rate',
            'Order Item Product Price', 'Order Item Quantity']
nominal_cols = ['Type', 'Delivery Status', 'Customer City', 'Customer Country',
                'Order Region', 'Order Status']
ordinal_cols = ['Profitability_Category','Order_Value_Category', 'Customer Segment']
binary_cols = ['Customer State']

In [3]:
# Numerical Pipeline
numerical_pipeline = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=3)),
    ('scaler_standard', StandardScaler()),
])

In [13]:
# Categorical Pipelines (OneHot, Ordinal, Binary)
onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

binary_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', BinaryEncoder())
])

Step4. Column Transformer to Assign columns to be processed

In [14]:
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, num_cols),
    ('nom', onehot_pipeline, nominal_cols),
    ('ord', ordinal_pipeline, ordinal_cols),
    ('bin', binary_pipeline, binary_cols)
])

Handling Imbalance 

In [ ]:

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Build pipeline with preprocessing, SMOTE, then classification
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('smote', SMOTE(random_state=4)),
    ('classifier', LogisticRegression(max_iter=200, random_state=4))
])

# Train the model
model_pipeline.fit(x_train, y_train)

# Predict and evaluate
y_pred = model_pipeline.predict(x_test)
print(classification_report(y_test, y_pred))

Step5. Model Definition and Training

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
# Reset index for consistency
x_train.reset_index(inplace=True, drop=True)
x_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

model = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=200, random_state=40))
])


model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [ ]:
px.histogram(data_frame= x_train, x= 'Order Profit Per Order')

Step6. Model Evaluation

In [ ]:
report = classification_report(y_test, y_pred, output_dict=True)
pd.DataFrame(report).transpose()

Step7. Cross Validation & Hyperparameter Tuning

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

model_cv = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=200, random_state=40))
])

cv_scores = cross_val_score(model_cv, x, y, cv=5, scoring='accuracy')
print("Cross Validation Scores:", cv_scores)
print("Average Accuracy:", cv_scores.mean())

param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2']
}

grid_search = GridSearchCV(model_cv, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x, y)

print("Best Parameters from GridSearch:", grid_search.best_params_)
print("Best CV Accuracy:", grid_search.best_score_)